In [51]:
import polars as pl
import random
import string
import pandas as pd

Data Structures

In [52]:
# series
a = pl.Series([1, 4, 4, 7, 7, 16, 25])
print(a)

b = pl.Series(
    "firstSeries",
    [1, 4, 7, 9, 16, 25])
print(b)

b2 = pl.Series("secondSeries",
               [36, 47])
print(b2)

# appending to a series
b = b.append(b2)
print(b)

# show unique values
print(b.unique())

# dataframe (eager execution)
df = pl.DataFrame(
    {
        "a": [1, 2, 3],
        "b": [4, 5, 6],
        "c": [7, 8, 9],
    }
)
print(df)

# describe dataframe
print(df.describe())

# convert to lazyframe (lazy execution)
plf1 = pl.DataFrame(
    {
        "fruits": ["Apple", "Peach", "Melon",
                   "Pineapple", "Orange"],
        "price": [22, 27, 25, 35, 29],  
        "quantity": [5, 7, 4, 3, 9],
    }
).lazy()

# conver pandas to lazyframe
pandas_df1 = pd.DataFrame(
    {
        "a": 
            ["alpha", "beta", "gamma",
             "delta", "eta"],
            "b": 
            [1, 2, 3, 4, 5]
    }
)
print(pandas_df1)

pandas_lazyframe = pl.LazyFrame(pandas_df1)
print(pandas_lazyframe)

shape: (7,)
Series: '' [i64]
[
	1
	4
	4
	7
	7
	16
	25
]
shape: (6,)
Series: 'firstSeries' [i64]
[
	1
	4
	7
	9
	16
	25
]
shape: (2,)
Series: 'secondSeries' [i64]
[
	36
	47
]
shape: (8,)
Series: 'firstSeries' [i64]
[
	1
	4
	7
	9
	16
	25
	36
	47
]
shape: (8,)
Series: 'firstSeries' [i64]
[
	1
	4
	7
	9
	16
	25
	36
	47
]
shape: (3, 3)
┌─────┬─────┬─────┐
│ a   ┆ b   ┆ c   │
│ --- ┆ --- ┆ --- │
│ i64 ┆ i64 ┆ i64 │
╞═════╪═════╪═════╡
│ 1   ┆ 4   ┆ 7   │
│ 2   ┆ 5   ┆ 8   │
│ 3   ┆ 6   ┆ 9   │
└─────┴─────┴─────┘
shape: (9, 4)
┌────────────┬─────┬─────┬─────┐
│ statistic  ┆ a   ┆ b   ┆ c   │
│ ---        ┆ --- ┆ --- ┆ --- │
│ str        ┆ f64 ┆ f64 ┆ f64 │
╞════════════╪═════╪═════╪═════╡
│ count      ┆ 3.0 ┆ 3.0 ┆ 3.0 │
│ null_count ┆ 0.0 ┆ 0.0 ┆ 0.0 │
│ mean       ┆ 2.0 ┆ 5.0 ┆ 8.0 │
│ std        ┆ 1.0 ┆ 1.0 ┆ 1.0 │
│ min        ┆ 1.0 ┆ 4.0 ┆ 7.0 │
│ 25%        ┆ 2.0 ┆ 5.0 ┆ 8.0 │
│ 50%        ┆ 2.0 ┆ 5.0 ┆ 8.0 │
│ 75%        ┆ 3.0 ┆ 6.0 ┆ 9.0 │
│ max        ┆ 3.0 ┆ 6.0 ┆ 9.0 │
└────────────

Data Extraction and Loading CSV

In [53]:
csv_path = "/Users/pepijnschouten/Desktop/Python_Scripts/" \
    "Python_Scripts_Books/Data_Engineering/Data_Engineering" \
        "_for_ML_Pipelines/Own_Files/3_Polars_Data_Wrangling/" \
            "data/winequality-red.csv"

# eager dataframe
polars_df = pl.read_csv(csv_path)

# lazy dataframe
polars_df_lazy = pl.scan_csv(csv_path)

Data Transformation in Polars

In [54]:
df = pl.DataFrame(
    {
        "A": [1, 2, 3],
        "B": [4, 5, 6],
        "C": [7, 8, 9],})

# select columns
print(df.select(["A",
                 (pl.col("B")*2).alias("NewCol_B")]))
print(df)

# create new column in df
df = df.with_columns(
    (pl.col("B")*2).alias("NewCol_B"))
print(df)

# filter context
conFilter = df.filter(pl.col("A") > 1)
print(conFilter)

# groupby context
df1 = pl.DataFrame(
    {
        "Region":
            ["CityBranch", "CampusBranch", "CityBranch",
             "SuburbBranch", "CampusBranch", "SuburbBranch",
             "CityBranch", "CampusBranch"],
        "Product":
            ["A", "B", "B",
             "A", "B", "A",
             "A", "B"],
        "Sales":
            [100, 200, 150, 300, 250, 400, 350, 180]
    })
print(df1)

groupbydf = (df1
             .group_by(["Region", "Product"])
             .agg(
                 [
                     pl.col("Sales").sum().alias("TotalSales"),
                     pl.col("Sales").max().alias("MaxSales"),
                    pl.count("Sales").alias("CountSales")
                 ]
             ))
print(groupbydf)

# groupby on lazyframe (collect())
df2 = df1.lazy()
print(df2)

groupbydf2 = (df2
             .group_by(["Region", "Product"])
             .agg(
                 [
                     pl.col("Sales").sum().alias("TotalSales"),
                     pl.col("Sales").max().alias("MaxSales"),
                    pl.count("Sales").alias("CountSales")
                 ]
             ))
print(groupbydf2.collect())

# strings to dates
data = {
        'date1': '2022-01-01',
        'date2': '2022-01-05',
        'date3': '2022-01-10',
        'date4': '2022-01-15',
        'date5': '2022-01-20',
        'date6': '2022-01-25',
        'date7': '2022-02-01',
        'date8': '2022-02-05',
        'date9': '2022-02-10',
        'date10': '2022-02-15'
        }
data_df = pl.DataFrame(
    {
        'Names': data.keys(),
        'Dates': data.values()
    }
)
print(data_df)

print(data_df.with_columns(
    pl.col("Dates").str.strptime(pl.Datetime, "%Y-%m-%d").alias("DateTime")))


shape: (3, 2)
┌─────┬──────────┐
│ A   ┆ NewCol_B │
│ --- ┆ ---      │
│ i64 ┆ i64      │
╞═════╪══════════╡
│ 1   ┆ 8        │
│ 2   ┆ 10       │
│ 3   ┆ 12       │
└─────┴──────────┘
shape: (3, 3)
┌─────┬─────┬─────┐
│ A   ┆ B   ┆ C   │
│ --- ┆ --- ┆ --- │
│ i64 ┆ i64 ┆ i64 │
╞═════╪═════╪═════╡
│ 1   ┆ 4   ┆ 7   │
│ 2   ┆ 5   ┆ 8   │
│ 3   ┆ 6   ┆ 9   │
└─────┴─────┴─────┘
shape: (3, 4)
┌─────┬─────┬─────┬──────────┐
│ A   ┆ B   ┆ C   ┆ NewCol_B │
│ --- ┆ --- ┆ --- ┆ ---      │
│ i64 ┆ i64 ┆ i64 ┆ i64      │
╞═════╪═════╪═════╪══════════╡
│ 1   ┆ 4   ┆ 7   ┆ 8        │
│ 2   ┆ 5   ┆ 8   ┆ 10       │
│ 3   ┆ 6   ┆ 9   ┆ 12       │
└─────┴─────┴─────┴──────────┘
shape: (2, 4)
┌─────┬─────┬─────┬──────────┐
│ A   ┆ B   ┆ C   ┆ NewCol_B │
│ --- ┆ --- ┆ --- ┆ ---      │
│ i64 ┆ i64 ┆ i64 ┆ i64      │
╞═════╪═════╪═════╪══════════╡
│ 2   ┆ 5   ┆ 8   ┆ 10       │
│ 3   ┆ 6   ┆ 9   ┆ 12       │
└─────┴─────┴─────┴──────────┘
shape: (8, 3)
┌──────────────┬─────────┬───────┐
│ Region       ┆ 

Dataframe Merging

In [55]:
# left join
df1 = pl.DataFrame({
'ID': [1, 2, 3, 4, 5],
'Name': ['Alice', 'Bob', 'Charlie', 'David', 'John'],
'DeptID': [101, 102, 103, 104, 103]})

df2 = pl.DataFrame({
'DeptID': [101, 102, 103],
'DeptName': ['HR', 'IT', 'Marketing']})

leftjoin = df1.join(
    df2,
    on = 'DeptID',
    how = 'left',
    coalesce=True)
print(leftjoin)

leftjoin2 = df1.join(
    df2,
    on = 'DeptID',
    how = 'left',
    coalesce=False)
print(leftjoin2)

shape: (5, 4)
┌─────┬─────────┬────────┬───────────┐
│ ID  ┆ Name    ┆ DeptID ┆ DeptName  │
│ --- ┆ ---     ┆ ---    ┆ ---       │
│ i64 ┆ str     ┆ i64    ┆ str       │
╞═════╪═════════╪════════╪═══════════╡
│ 1   ┆ Alice   ┆ 101    ┆ HR        │
│ 2   ┆ Bob     ┆ 102    ┆ IT        │
│ 3   ┆ Charlie ┆ 103    ┆ Marketing │
│ 4   ┆ David   ┆ 104    ┆ null      │
│ 5   ┆ John    ┆ 103    ┆ Marketing │
└─────┴─────────┴────────┴───────────┘
shape: (5, 5)
┌─────┬─────────┬────────┬──────────────┬───────────┐
│ ID  ┆ Name    ┆ DeptID ┆ DeptID_right ┆ DeptName  │
│ --- ┆ ---     ┆ ---    ┆ ---          ┆ ---       │
│ i64 ┆ str     ┆ i64    ┆ i64          ┆ str       │
╞═════╪═════════╪════════╪══════════════╪═══════════╡
│ 1   ┆ Alice   ┆ 101    ┆ 101          ┆ HR        │
│ 2   ┆ Bob     ┆ 102    ┆ 102          ┆ IT        │
│ 3   ┆ Charlie ┆ 103    ┆ 103          ┆ Marketing │
│ 4   ┆ David   ┆ 104    ┆ null         ┆ null      │
│ 5   ┆ John    ┆ 103    ┆ 103          ┆ Marketing │
└──

Advanced Operations

In [56]:
size = 10
df = pl.DataFrame({
    'A': [random.choice([random.randint(0, 100), None]) for _ in range(size)],
    'B': [random.choice([random.randint(0, 100), None]) for _ in range(size)],
    'C': [random.choice([random.randint(0, 100), None]) for _ in range(size)]
})
print(df)

# identify null values
missing_data = df.with_columns(pl.all().is_null())
print(missing_data)

# identify null values in one column
missing_col = df.filter(
    pl.col("A").is_null())
print(missing_col)

# identify unique values
unique_A = df.unique("A")
print(unique_A) 

# with groupby
unique_A_groupby = (df
                    .group_by("A")
                    .agg(pl.count('A')
                         .alias("Count")))
print(unique_A_groupby)

# pivot melt example
pivotdata = df.pivot(values="B",
                     index="A",
                     on="C",
                     aggregate_function="first")
print(pivotdata)

shape: (10, 3)
┌──────┬──────┬──────┐
│ A    ┆ B    ┆ C    │
│ ---  ┆ ---  ┆ ---  │
│ i64  ┆ i64  ┆ i64  │
╞══════╪══════╪══════╡
│ 15   ┆ 23   ┆ 3    │
│ null ┆ 64   ┆ 59   │
│ 25   ┆ null ┆ 25   │
│ null ┆ 26   ┆ null │
│ 17   ┆ 94   ┆ 71   │
│ null ┆ 0    ┆ null │
│ null ┆ 7    ┆ null │
│ 9    ┆ null ┆ null │
│ 47   ┆ 23   ┆ null │
│ 95   ┆ 56   ┆ null │
└──────┴──────┴──────┘
shape: (10, 3)
┌───────┬───────┬───────┐
│ A     ┆ B     ┆ C     │
│ ---   ┆ ---   ┆ ---   │
│ bool  ┆ bool  ┆ bool  │
╞═══════╪═══════╪═══════╡
│ false ┆ false ┆ false │
│ true  ┆ false ┆ false │
│ false ┆ true  ┆ false │
│ true  ┆ false ┆ true  │
│ false ┆ false ┆ false │
│ true  ┆ false ┆ true  │
│ true  ┆ false ┆ true  │
│ false ┆ true  ┆ true  │
│ false ┆ false ┆ true  │
│ false ┆ false ┆ true  │
└───────┴───────┴───────┘
shape: (4, 3)
┌──────┬─────┬──────┐
│ A    ┆ B   ┆ C    │
│ ---  ┆ --- ┆ ---  │
│ i64  ┆ i64 ┆ i64  │
╞══════╪═════╪══════╡
│ null ┆ 64  ┆ 59   │
│ null ┆ 26  ┆ null │
│ null ┆ 0   ┆ nul

SQL Interaction

In [72]:
df = pl.DataFrame({
    'id': [i for i in range(size)],
    'name': [''.join(random.choices(string.ascii_lowercase, k=10))
             for _ in range(size)],
    'description': [''.join(random.choices(string.ascii_lowercase, k=20))
                    for _ in range(size)],
    'price': [f'{random.randint(0, 100)}.{random.randint(0, 99)}'
              for _ in range(size)],
    'quantity': [str(random.randint(0, 100))
                 for _ in range(size)]
})

print(df)

# convert to numerical values
df = df.with_columns(
    [
        pl.col("price").str.to_decimal(),
        pl.col("quantity").str.to_integer()
    ]
)
print(df)

# create SQL context
pl_sql = pl.SQLContext(
    random_df = df)

query = """
    SELECT
        price,
        count(price) as total_quantity,
        sum(price) as total_price
    FROM
        random_df
    WHERE
        quantity > 10
    GROUP BY
        price
    ORDER BY
        total_price DESC

"""
print(pl_sql.execute(query,
                     eager=True))

shape: (10, 5)
┌─────┬────────────┬──────────────────────┬────────┬──────────┐
│ id  ┆ name       ┆ description          ┆ price  ┆ quantity │
│ --- ┆ ---        ┆ ---                  ┆ ---    ┆ ---      │
│ i64 ┆ str        ┆ str                  ┆ str    ┆ str      │
╞═════╪════════════╪══════════════════════╪════════╪══════════╡
│ 0   ┆ pgzqkszyux ┆ jijcscietouoibmanqgm ┆ 92.64  ┆ 55       │
│ 1   ┆ kcyedxldfx ┆ zrvawmgcdokvqvvcyplc ┆ 79.19  ┆ 97       │
│ 2   ┆ krwbdtqdvb ┆ gseoyvcoykmhylaadnbn ┆ 54.44  ┆ 86       │
│ 3   ┆ oefqwkoaja ┆ lalulrxwoufiexotkxyy ┆ 0.66   ┆ 41       │
│ 4   ┆ nqqxjfmwwk ┆ iebeihdeocjonayvyerz ┆ 77.28  ┆ 97       │
│ 5   ┆ rduiaqsgur ┆ ldmherwfvxwginsnmief ┆ 28.66  ┆ 35       │
│ 6   ┆ efatgcvnqw ┆ rktflxrcavvzknxpkulz ┆ 92.91  ┆ 48       │
│ 7   ┆ osaaidmphp ┆ nhlssnzcoqouhmyhqzrs ┆ 100.33 ┆ 66       │
│ 8   ┆ bacrppxpso ┆ fbefmvtkopphidzfjppp ┆ 68.21  ┆ 49       │
│ 9   ┆ iyekjmxbsh ┆ skynectqagidhndiywdm ┆ 46.18  ┆ 24       │
└─────┴────────────┴─────